In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsRegressor
from lightgbm import LGBMRegressor
from xgboost import XGBRegressor
from scipy import optimize, stats                     
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from lightgbm import LGBMRegressor
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.model_selection import StratifiedKFold, KFold
from sklearn.metrics import mean_squared_error


X_train = pd.read_csv('../input/competitive-data-science-predict-future-sales/sales_train.csv')
X_test = pd.read_csv('../input/competitive-data-science-predict-future-sales/test.csv')
X_train.head()

In [ ]:
X_train.drop(['date_block_num','item_price'], axis=1, inplace=True)
X_train['date'] = pd.to_datetime(X_train['date'], dayfirst=True)
X_train['date'] = X_train['date'].apply(lambda x: x.strftime('%Y-%m'))
X_train.head()

In [ ]:
X = X_train.groupby(['date','shop_id','item_id']).sum()
X = X.pivot_table(index=['shop_id','item_id'], columns='date', values='item_cnt_day', fill_value=0)
X.reset_index(inplace=True)
X.head()

In [ ]:
X_test = pd.merge(X_test, X, on=['shop_id','item_id'], how='left')
X_test.drop(['2013-01'], axis=1, inplace=True)
X_test = X_test.fillna(0)
X_test.head()

In [ ]:
Y = X['2015-10'].values
X_T = X.drop(['2015-10'], axis = 1)
X_test = X_test
x_train, x_test, y_train, y_test = train_test_split( X_T, Y, test_size=0.20, random_state=1)


In [ ]:
lin_reg = LinearRegression(normalize=True)
lin_reg.fit(x_train,y_train)
print(lin_reg.intercept_)

In [ ]:
pred = lin_reg.predict(x_test)
plt.scatter(y_test, pred)

In [ ]:
RF_reg = RandomForestRegressor(n_estimators = 100)
RF_reg.fit(x_train, y_train)
pred = RF_reg.predict(x_test)

In [ ]:
XGB_reg = XGBRegressor(max_depth=16,n_estimators=200,seed=1)
XGB_reg.fit(x_train,y_train)

In [ ]:
from sklearn import metrics
from sklearn.model_selection import cross_val_score
import numpy as np
def cross_val(model):
    pred = cross_val_score(model, X, Y, cv=10)
    return pred.mean()

def print_evaluate(true, predicted):  
    mae = metrics.mean_absolute_error(true, predicted)
    mse = metrics.mean_squared_error(true, predicted)
    rmse = np.sqrt(metrics.mean_squared_error(true, predicted))
    r2_square = metrics.r2_score(true, predicted)
    print('MAE:', mae)
    print('MSE:', mse)
    print('RMSE:', rmse)
    print('R2 Square', r2_square)
    print('__________________________________')
    
def evaluate(true, predicted):
    mae = metrics.mean_absolute_error(true, predicted)
    mse = metrics.mean_squared_error(true, predicted)
    rmse = np.sqrt(metrics.mean_squared_error(true, predicted))
    r2_square = metrics.r2_score(true, predicted)
    return mae, mse, rmse, r2_square

In [ ]:
test_pred = lin_reg.predict(x_test)
train_pred = lin_reg.predict(x_train)
print_evaluate(y_test, test_pred)
print_evaluate(y_train, train_pred)

In [ ]:
test_pred = RF_reg.predict(x_test)
train_pred = RF_reg.predict(x_train)
print_evaluate(y_test, test_pred)
print_evaluate(y_train, train_pred)

In [ ]:
test_pred = XGB_reg.predict(x_test)
train_pred = XGB_reg.predict(x_train)
print_evaluate(y_test, test_pred)
print_evaluate(y_train, train_pred)

In [ ]:
RF_reg = RandomForestRegressor(n_estimators = 100)
RF_reg.fit(X, Y)
pred = RF_reg.predict(X_test)

In [ ]:
output = pd.DataFrame({'Id': X_test.index,'item_cnt_month': pred})
output.to_csv('submission.csv', index=False)

In [ ]:
output.shape